In [6]:
search_for = 118
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.008244991302490234
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 382114711
type: [1, 1, 1, 1, 118] 118
cases of this type: 193877776
10000 30.06659201474506
20000 31.105355185365298
30000 31.64388565657076
40000 32.750283795083284
50000 129.42133821958177
60000 167.85755742112417
70000 137.09342328762713
80000 144.63086560214057
90000 152.13622932191737
100000 137.69721127094917
110000 148.94307602327848
120000 134.10359946926016
130000 164.85255739131986
140000 144.68206156318737
150000 171.3724957437516
160000 172.29116387714603
170000 167.5483564834415
180000 200.65247160956656
190000 215.6658655632454
200000 280.3029175303387
210000 272.651598019159
220000 279.5876281368468
230000 260.92675897781106
240000 273.3117406817293
250000 280.66644073585667
260000 262.6751194579716
270000 274.7383311586877
280000 268.88597143847016
290000 266.092853464777
300000 262.0723262558586
310000 274.76973

3070000 256.4054101122156
3080000 242.5495864387461
3090000 266.84222842674023
3100000 248.67174577241778
3110000 266.04059498063384
3120000 242.3544129347255
3130000 266.5527952200478
3140000 248.93197606935766
3150000 254.27495962804934
3160000 263.7758543081167
3170000 247.689033534655
3180000 268.4599196565703
3190000 233.57310299224764
3200000 273.6538164651976
3210000 236.12668044823087
3220000 261.7421518312498
3230000 277.8467540651392
3240000 274.5515814832842
3250000 175.69748781110897
3260000 81.5787164436887
3270000 85.05606976215594
3280000 81.94727346169896
3290000 78.40141012613961
3300000 65.7121175281631
3310000 65.58249811250946
3320000 67.02281882963312
3330000 65.26147033855828
3340000 66.1837252677444
3350000 65.36236638802306
3360000 197.3243594670687
3370000 329.33918026149485
3380000 298.63295084239985
3390000 282.69510021501424
3400000 243.96454074881493
3410000 295.68936463741403
3420000 260.1064414680922
3430000 302.72553813601587
3440000 273.447757317328
345

6180000 299.45273940452364
6190000 286.2832488605672
6200000 284.4417803914659
6210000 296.64129150864335
6220000 292.82331327940005
6230000 257.7162101912354
6240000 266.1516457125
6250000 267.50238798111536
6260000 272.4163322412571
6270000 270.2442060393606
6280000 264.0035186317472
6290000 265.3545063261384
6300000 265.8285295606461
6310000 251.03889941240035
6320000 271.97339394647884
6330000 263.61177217533964
6340000 268.7827875657064
6350000 250.8386970770922
6360000 285.1181642371043
6370000 273.99313741253985
6380000 255.2320118855204
6390000 281.7521075914778
6400000 263.09655654385887
6410000 288.81116472122596
6420000 261.30590877276444
6430000 285.05213354997875
6440000 266.2746317752934
6450000 264.05387912367627
6460000 273.72034753872623
6470000 242.34089833708057
6480000 274.2387782624456
6490000 240.74831897757667
6500000 323.506454743923
6510000 127.39318526965492
6520000 82.72698521922463
6530000 81.2432752268877
6540000 80.7738740653679
6550000 82.91647614118523
6

9290000 245.67367454582788
9300000 269.0608779373407
9310000 256.28635834126317
9320000 245.0783928974024
9330000 245.5951125052731
9340000 265.7180057997282
9350000 253.83289036809114
9360000 255.9805691219175
9370000 258.0654004512902
9380000 261.7751738787136
9390000 241.36157451622995
9400000 253.58197011371186
9410000 288.6388373311388
9420000 262.8030300758563
9430000 260.794414557429
9440000 247.50463559796606
9450000 274.7488315753265
9460000 252.633343292791
9470000 270.7873630385074
9480000 283.63451181253174
9490000 285.1644234644602
9500000 288.51292004207187
9510000 291.8377566610748
9520000 272.718410610235
9530000 266.3382333035449
9540000 264.13341763561203
9550000 257.1583773548131
9560000 266.82705219136466
9570000 264.8544338280383
9580000 266.7832277013246
9590000 261.0201325195085
9600000 263.5087272815121
9610000 243.84954892283682
9620000 272.8126841062816
9630000 255.80339475294957
9640000 266.9857116962452
9650000 250.24027270284853
9660000 272.7025555899289
96

12310000 32.194128303773624
12320000 31.908530761309812
12330000 31.935476141437277
12340000 32.1766100075593
12350000 31.97542350743516
12360000 32.13504991879519
12370000 202.2748652789909
12380000 294.3840026641682
12390000 289.45555493714164
12400000 206.92034952311883
12410000 238.91375714626218
12420000 245.6014758592878
12430000 261.7359494406317
12440000 278.50720615256887
12450000 248.25752071616827
12460000 280.07485953403335
12470000 257.861237362569
12480000 274.590030905183
12490000 261.4574428305699
12500000 273.27511918255817
12510000 281.84243330731766
12520000 271.45723947596133
12530000 279.1246712585128
12540000 273.6900213294372
12550000 281.496407890741
12560000 276.95057319176004
12570000 261.2603049265133
12580000 268.4923468214013
12590000 257.09777140696673
12600000 254.00461911603077
12610000 262.0838727703419
12620000 250.54965408448456
12630000 265.24221840423326
12640000 268.45624766049025
12650000 269.8740989211535
12660000 270.61864773694754
12670000 260.

...

369400000 0.8606728702893852
369410000 0.8553277891918951
369420000 0.8589622391474313
369430000 0.8564585295560757
369440000 0.8635039243935082
369450000 0.8572712882712947
369460000 0.8518110011352897
369470000 0.8496756536843776
369480000 0.8432159642099539
369490000 0.841543840459327
369500000 0.8478797409289346
369510000 0.8534137550300623
369520000 0.8461315911984246
369530000 0.8444998889701234
369540000 0.8396156677858632
369550000 0.8678844658315181
369560000 0.8520099240371585
369570000 0.849409056630585
369580000 0.84959629423062
369590000 0.8399719894967675
369600000 0.8370035266735091
369610000 0.8338969560124675
369620000 0.8358304006744557
369630000 0.838697969464653
369640000 0.8401073052639365
369650000 0.8356322404500314
369660000 0.8377547601581083
369670000 0.8399805977758766
369680000 0.8405250938252806
369690000 0.8350904018096859
369700000 0.8400642861959735
369710000 0.8283630751763185
369720000 0.8289064046674967
369730000 0.8327963570183516
369740000 0.8376092

372150000 1.3958175987658832
372160000 1.1971176166804434
372170000 3.838683052805013
372180000 3.489551508483523
372190000 3.9042957305477657
372200000 3.832996913500051
372210000 2.9361473992506197
372220000 1.9751067374305522
372230000 4.152562117522955
372240000 4.014866974245449
372250000 4.0418213649437424
372260000 4.247628653325399
372270000 3.886949119730386
372280000 4.0574930363980535
372290000 3.6153141040482786
372300000 1.6328573598417375
372310000 2.8549781642865537
372320000 3.705363995224039
372330000 3.686023341661553
372340000 3.4903239865485425
372350000 2.8391378391488655
372360000 0.8391675510856641
372370000 2.5100824349100987
372380000 3.618950219650123
372390000 3.606070566923811
372400000 3.65275636708053
372410000 3.0407935638290113
372420000 1.941909535797139
372430000 3.355185812847912
372440000 4.077495426338249
372450000 3.8864186557425464
372460000 4.085896055185199
372470000 4.155538860132389
372480000 3.9368481267168325
372490000 4.008448881029805
3725

375030000 0.6419768245397409
375040000 1.1108438940617442
375050000 1.7791273309979108
375060000 2.9369378062185576
375070000 2.72555721712182
375080000 2.6708483478562237
375090000 2.606417318999555
375100000 2.6075445819485186
375110000 2.61904193289266
375120000 2.6804393810205327
375130000 3.1134009153615834
375140000 3.3088067584728864
375150000 2.8411755588756007
375160000 2.897288308800936
375170000 2.7190967591299016
375180000 2.792237079626554
375190000 2.960334564065059
375200000 2.783206395164953
375210000 3.0339997764729394
375220000 1.6152669352074265
375230000 0.7474888005702098
375240000 0.6542713988669845
375250000 1.427036260946274
375260000 2.9520864027392073
375270000 2.7248905911032555
375280000 2.9386682238671975
375290000 2.8951788035293617
375300000 2.6721097187333043
375310000 2.7034818804723026
375320000 2.782137266626305
375330000 3.1187058674664034
375340000 1.5308135833174983
375350000 0.5341767144090268
375360000 0.5282319717724324
375370000 0.5233702029275

377880000 1.3648551953530974
377890000 1.7222323158319195
377900000 0.6226962241633998
377910000 0.477660526235885
377920000 1.6686043718094228
377930000 1.4410726902555957
377940000 0.721957278611011
377950000 1.2636998072291215
377960000 1.5802774314379031
377970000 1.6422437242477204
377980000 1.7400811803198255
377990000 1.6266749776366751
378000000 1.6390280373090769
378010000 1.775631953760147
378020000 1.678073962014993
378030000 1.7940048708855574
378040000 1.6866873793206811
378050000 1.5700775326327947
378060000 1.5049195092052021
378070000 0.8308704429402749
378080000 1.106897527650469
378090000 1.5597645523088706
378100000 1.3487037818033694
378110000 0.3592718952587313
378120000 1.0637200133982632
378130000 1.608263682083686
378140000 1.1612795518464911
378150000 0.6776267201405763
378160000 1.2735200306772392
378170000 1.454101060209877
378180000 1.6433150234867135
378190000 1.6826273282955289
378200000 1.6259145210835602
378210000 1.6297357422056131
378220000 1.707582433

380720000 0.5781261435045136
380730000 0.5558802241975798
380740000 0.57493689236782
380750000 0.5348778216184046
380760000 0.5249188793875442
380770000 0.41052445714666447
380780000 0.12003767320566708
380790000 0.3544864257183075
380800000 0.5020049868929386
380810000 0.5018172936413023
380820000 0.5512941504305005
380830000 0.5166216016177138
380840000 0.5228515608510905
380850000 0.533030717432499
380860000 0.25469109621008235
380870000 0.2749358030417429
380880000 0.5396387483021683
380890000 0.4214356495217681
380900000 0.24505358482331038
380910000 0.4361473031339646
380920000 0.5037357132943868
380930000 0.48951268460114133
380940000 0.4798137819308705
380950000 0.4959201584422588
380960000 0.48457439276773395
380970000 0.43925917980848417
380980000 0.20903780871462824
380990000 0.10140181976496511
381000000 0.4603390639585058
381010000 0.4509517866754731
381020000 0.44403278123505907
381030000 0.46465232674739754
381040000 0.4323777589148283
381050000 0.4391837513187329
381060